![GoogleNet](https://github.com/user-attachments/assets/5b815a8d-e8af-4139-9759-ec4b2727e167)


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
batch_size= 100
learning_rate = 0.0002
num_epoch = 200

In [3]:
train_d = dset.CIFAR10(root = "../", train = True, transform = transforms.Compose([
                                                transforms.Resize(256),
                                                transforms.RandomResizedCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
]), download = True)
test_d = dset.CIFAR10(root = "../", train = False, transform = transforms.Compose([
                                                transforms.Resize(256),
                                                transforms.RandomResizedCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor()
]), download = True)
train_loader = data.DataLoader(dataset = train_d, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader = data.DataLoader(dataset = test_d, batch_size = batch_size, shuffle = False, drop_last = True)

100%|██████████| 170498071/170498071 [00:24<00:00, 6949399.15it/s]


Extracting ../cifar-10-python.tar.gz to ../
Files already downloaded and verified


- Base Model
  - 인셉션 모듈을 구성하는 4가지 연산을 함수로 정의

In [4]:
def conv_1(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, 1, 1),
        nn.ReLU(),
    )
    return model

def conv_1_3(in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim, 3, 1, 1),
        nn.ReLU()
    )
    return model

def conv_1_5(in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim, 5, 1, 2),
        nn.ReLU()
    )
    return model

def max_3_1(in_dim, out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(3, 1, 1),
        nn.Conv2d(in_dim, out_dim, 1, 1),
        nn.ReLU()
    )
    return model

- Inception Module
  - 앞서 정의한 연산으로 인셉션 모듈 만들기

In [5]:
class inception_module(nn.Module):
    def __init__(self, in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool):
        super(inception_module, self).__init__()
        self.conv_1 = conv_1(in_dim, out_dim_1)
        self.conv_1_3 = conv_1_3(in_dim, mid_dim_3, out_dim_3)
        self.conv_1_5 = conv_1_5(in_dim, mid_dim_5, out_dim_5)
        self.max_3_1 = max_3_1(in_dim, pool)
    
    def forward(self, x):
        out_1 = self.conv_1(x)
        out_2 = self.conv_1_3(x)
        out_3 = self.conv_1_5(x)
        out_4 = self.max_3_1(x)
        output = torch.cat([out_1, out_2, out_3, out_4], 1)
        print(output.shape)
        return output

![GoogleNet_sheet](https://github.com/user-attachments/assets/b5f2a4f8-ae6d-4bb2-aa1a-b58036739024)
